In [ ]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

def load_data(ratings_path: str, users_path: str, movies_path: str):
    """
    Load MovieLens 1M dataset from given file paths.
    The .dat files use '::' as separator and have no header row.
    Returns Pandas DataFrames: ratings_df, users_df, movies_df.
    """
    # Define column names for each file as per the dataset description
    ratings_cols = ['UserID', 'MovieID', 'Rating', 'Timestamp']
    users_cols   = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']
    movies_cols  = ['MovieID', 'Title', 'Genres']
    # Load the ratings data (UserID::MovieID::Rating::Timestamp)
    ratings_df = pd.read_csv(ratings_path, sep='::', engine='python',
                              names=ratings_cols,
                              dtype={'UserID': int, 'MovieID': int, 'Rating': int, 'Timestamp': int})
    # Load the users data (UserID::Gender::Age::Occupation::Zip-code)
    users_df = pd.read_csv(users_path, sep='::', engine='python',
                            names=users_cols,
                            dtype={'UserID': int, 'Gender': str, 'Age': int, 'Occupation': int, 'Zip-code': str})
    # Load the movies data (MovieID::Title::Genres)
    movies_df = pd.read_csv(movies_path, sep='::', engine='python',
                             names=movies_cols, encoding='latin1',
                             dtype={'MovieID': int, 'Title': str, 'Genres': str})
    return ratings_df, users_df, movies_df

def preprocess_data(ratings_df: pd.DataFrame, users_df: pd.DataFrame, movies_df: pd.DataFrame):
    """
    Preprocess the MovieLens data:
    - Handle missing values (if any) by removing incomplete entries.
    - Normalize ratings from 1-5 to 0-1 scale.
    - Parse the genre strings into lists of genres for each movie.
    Returns the cleaned and modified DataFrames.
    """
    # 1. Remove any missing values
    ratings_df = ratings_df.dropna()
    users_df   = users_df.dropna()
    movies_df  = movies_df.dropna()
    # 2. Normalize ratings to [0, 1] range (1 -> 0.0, 5 -> 1.0)
    ratings_df['Rating_norm'] = (ratings_df['Rating'] - 1) / 4.0
    # 3. Parse genres into a list for each movie (split by '|')
    movies_df['Genres'] = movies_df['Genres'].apply(
        lambda x: x.split('|') if isinstance(x, str) else [])
    return ratings_df, users_df, movies_df

def split_data(ratings_df: pd.DataFrame, test_ratio: float = 0.2):
    """
    Split ratings data into training and test sets for evaluation.
    Ensures each user has at least one rating in train and one in test (if possible).
    Returns (train_df, test_df).
    """
    # Group by user and split each user's ratings
    train_list = []
    test_list = []
    for user_id, group in ratings_df.groupby('UserID'):
        # Shuffle the user's ratings to randomize selection for test
        user_ratings = group.sample(frac=1.0, random_state=42)
        # Determine number of test ratings for this user
        test_count = max(1, int(len(user_ratings) * test_ratio))
        if len(user_ratings) <= 1:
            # If the user has only 1 rating, keep it in training (no test data for this user)
            train_ratings = user_ratings
            test_ratings = pd.DataFrame(columns=ratings_df.columns)
        else:
            # Split the user's ratings into test and train portions
            test_ratings = user_ratings.iloc[:test_count]
            train_ratings = user_ratings.iloc[test_count:]
            # Ensure train is not empty (if test_ratio rounds up to all ratings)
            if train_ratings.empty:
                train_ratings = test_ratings.iloc[:1]
                test_ratings = test_ratings.iloc[1:]
        train_list.append(train_ratings)
        test_list.append(test_ratings)
    # Concatenate all users' splits and reset index
    train_df = pd.concat(train_list).reset_index(drop=True)
    test_df  = pd.concat(test_list).reset_index(drop=True)
    return train_df, test_df

# ------------------ Model Training ---------------------
def train_recommender(train_df: pd.DataFrame):
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(train_df[['UserID', 'MovieID', 'Rating']], reader)
    trainset = data.build_full_trainset()
    algo = SVD(random_state=42)
    algo.fit(trainset)
    # Build item latent factor matrix: raw MovieID -> latent vector
    item_factors = {int(trainset.to_raw_iid(iid)): algo.qi[iid] for iid in trainset.all_items()}
    return algo, trainset, item_factors

# --------------- Initial Recommendations ---------------
def generate_candidates(model, trainset, user_id: int, movies_df: pd.DataFrame, candidate_pool_size: int = 10):
    all_ids = set(movies_df['MovieID'])
    try:
        inner_uid = trainset.to_inner_uid(user_id)
        seen_raw = {int(trainset.to_raw_iid(i)) for i, _ in trainset.ur[inner_uid]}
    except ValueError:
        seen_raw = set()
    pool = []
    for mid in all_ids - seen_raw:
        est = model.predict(user_id, mid).est
        pool.append((mid, est))
    pool.sort(key=lambda x: x[1], reverse=True)
    return pool[:candidate_pool_size]

# -------- Exhaustive Re-ranking (All Sequences) --------
def exhaustive_rerank(pool: list, item_factors: dict, movies_df: pd.DataFrame,
                     N: int = 5, w_rel: float = 0.6, w_div: float = 0.2, w_fair: float = 0.2):
    """
    Evaluate all possible combinations of length N from the pool
    and return the combination with maximum total utility.
    Utility of combination is:
      w_rel * sum(rel_i) + w_div * diversity + w_fair * fairness
    where diversity = 1 - avg pairwise cosine similarity among items,
    fairness = coverage_ratio of genres in the combination.
    """
    all_genres = set(g for genres in movies_df['Genres'] for g in genres)
    best_combo, best_score, best_total_rel, best_diversity, best_fair, best_genres = None, -np.inf, -np.inf, -np.inf, -np.inf, 0

    # Generate all possible combinations
    for combo in combinations(pool, N):
        mids = [mid for mid, _ in combo]
        ests = [est for _, est in combo]

        # Calculate overall relevance
        total_rel = sum((est - 1) / 4.0 for est in ests) / N

        # Calculating diversity: average pairwise cosine similarity
        vectors = [item_factors.get(mid) for mid in mids if mid in item_factors]
        if len(vectors) >= 2:
            stack = np.vstack(vectors)
            sim_matrix = cosine_similarity(stack)
            # Get the upper triangular matrix (excluding the diagonal)
            upper_tri = sim_matrix[np.triu_indices(len(vectors), k=1)]
            avg_sim = np.mean(upper_tri) if len(upper_tri) > 0 else 0
            diversity = 1 - avg_sim
        else:
            diversity = 0.0

        # Computational Fairness: Coverage
        genres_list = [set(movies_df[movies_df['MovieID'] == mid]['Genres'].iloc[0]) for mid in mids]
        covered = set().union(*genres_list)
        fair = len(covered) / len(all_genres) if all_genres else 0.0
        genres = len(covered)

        # Calculating overall utility
        total_utility = w_rel * total_rel + w_div * diversity + w_fair * fair

        # Update best combination
        if total_utility > best_score:
            best_score = total_utility
            best_combo = combo
            best_total_rel = total_rel
            best_diversity = diversity
            best_fair = fair
            best_genres = genres

    # If the best combination is found, sort it in descending order of relevance score and return it
    if best_combo is not None:
        best_seq = [mid for mid, _ in sorted(best_combo, key=lambda x: x[1], reverse=True)]
        return best_seq, best_total_rel, best_diversity, best_fair, best_genres
    else:
        return None

# ---------------- Evaluation Function ------------------
def evaluate_recs(model, trainset, item_factors, test_df: pd.DataFrame, users_df: pd.DataFrame, movies_df: pd.DataFrame,
                  N: int = 5, candidate_pool_size: int = 10, w_rel: float = 0.6, w_div: float = 0.2, w_fair: float = 0.2):
    precisions, recalls, ilds = [], [], []
    count = 0
    for user in test_df['UserID'].unique():
        print(user)
        count = count + 1
        if count > 10: # only output first 10 persons
          break
        pool = generate_candidates(model, trainset, user, movies_df, candidate_pool_size)
        best_seq, best_total_rel, best_diversity, best_fair, best_genres = exhaustive_rerank(pool, item_factors, movies_df, N, w_rel, w_div, w_fair)
        print("Best Sequence:", best_seq)
        print("Best Total Relevance:", best_total_rel)
        print("Best Diversity:", best_diversity)
        print("Best Fairness:", best_fair)
        print("Best Genres:", best_genres)



In [ ]:
# 1. Load the data
ratings_df, users_df, movies_df = load_data('ml-1m/ratings.dat', 'ml-1m/users.dat', 'ml-1m/movies.dat')

# 2. Preprocess the data
ratings_df, users_df, movies_df = preprocess_data(ratings_df, users_df, movies_df)

# 3. Split into training and test sets for evaluation
train_df, test_df = split_data(ratings_df, test_ratio=0.2)

# 4. Train the recommendation model on the training set
model, trainset, item_factors = train_recommender(train_df)

# 5. Evaluate with exhaustive reranking
evaluate_recs(
    model, trainset, item_factors, test_df,
    users_df, movies_df, N=5, candidate_pool_size=20,
    w_rel = 0.6, w_div = 0.2, w_fair = 0.2
)
